# Lab 9 Group 21



### New Vuln 1 :Improper Password validation and secure-by-default handling.
SampleNetworkServer.py does not properly handle commands chained to invalid AUTH and Logout requests. An attacker can submit a invalid password to the AUTH command and still be allowed to execute all other commands such as SET_DEGF or UPDATE_TEMP that would impact the legitimate client. This is a direct impact on the confidentiality of the system as it can be access by anyone.
### New Vuln 2: Inaccurate Incubator Temperature Reporting on unit change
Both updateInfTemp and updateIncTemp use the same calculation for temperature regardless of what units the different systems are using. This has a direct impact on the integrity of the reporting of temperatures.
*SampleNetworkServer.py Lines 157 & 165*
```
self.infTemps.append(self.infTherm.getTemperature()-273)
self.incTemps.append(self.incTherm.getTemperature()-273)
```
This causes inconsistencies in the reporting of temperatures. This not only can cause unintended temperature changes in the incubator, but also causes the graph to not display the lines, as the values fall out of the visibile range, which would make it hard for a medical professional to monitor.

### New Vuln 3: Lack of ability to confirm set units.
There is no public facing command to retrieve the units set despite having the commands to set the units at any time. Temperatures without a unit give little context to a medical professional. An attacker could change the units without the other party knowing using Vulns 1 and 2 and cause the medical professional to respond inappropriately as they have lost integrity of their configurations and the availability they should have gained by being able to set they dynamically. Further the command to set the units does not return any sort of confirmation or status leaving those callers often hanging awaiting a response and/or unsure of the outcome. This gaps This could lead to clients DoSing the client with the SET_DEG commands.

In [45]:
#Exploit
import unittest
import socket

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

class TestExploits(unittest.TestCase):
    # Vuln 1 Exploit Test
    def test_bad_password(self):
        s_rl = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
        input_request=b'AUTH basd;GET_TEMP'
        s_rl.sendto(input_request, ("127.0.0.1", 23456))
        output_responce, real_server_addr = s_rl.recvfrom(1024)
        output_responce=str(output_responce.strip(),"utf8")
        s_rl.close()
        # print(output_responce)
        self.assertFalse(is_number(output_responce),msg="Received Valid Temp when should have been rejected for invalid auth.")
    # Vuln 2 Unit change Test
    def test_unit_change(self):
        s_rl = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
        request_1=b'AUTH basd;GET_TEMP'
        s_rl.sendto(request_1, ("127.0.0.1", 23457))
        output_responce, real_server_addr = s_rl.recvfrom(1024)
        deg_k=float(output_responce.strip())
        print(f"deg kelvin={deg_k}")
        input_request=b'AUTH basd;SET_DEGF'
        s_rl.sendto(input_request, ("127.0.0.1", 23457))
        # output_responce, real_server_addr = s_rl.recvfrom(1024)
        s_rl.sendto(request_1, ("127.0.0.1", 23457))
        output_responce, real_server_addr = s_rl.recvfrom(1024)
        deg_f=float(output_responce.strip())
        print(f"deg f={deg_f}")
        request_3=b'AUTH basd;SET_DEGC'
        s_rl.sendto(request_3, ("127.0.0.1", 23457))
        s_rl.sendto(request_1, ("127.0.0.1", 23457))
        output_responce, real_server_addr = s_rl.recvfrom(1024)
        s_rl.close()
        deg_c=float(output_responce.strip())
        print(f"deg celcius={deg_c}")
        self.assertNotAlmostEqual(deg_c,deg_f,places=0,msg="Reported Temp in F and C should not be the same.")
        self.assertNotAlmostEqual(deg_k,deg_c,places=0,msg="Reported Temp in K and C should not be the same.")
        self.assertNotAlmostEqual(deg_k,deg_f,places=0,msg="Reported Temp in K and F should not be the same.")



res = unittest.main(argv=[''], verbosity=3, exit=False)

test_bad_password (__main__.TestExploits) ... FAIL
test_unit_change (__main__.TestExploits) ... 

deg kelvin=74.267842
deg f=165.682984


FAIL

FAIL: test_bad_password (__main__.TestExploits)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/tc/3ljmtg750xz6n4_dy5k9l0mm0000gn/T/ipykernel_63586/781501239.py", line 22, in test_bad_password
    self.assertFalse(is_number(output_responce),msg="Received Valid Temp when should have been rejected for invalid auth.")
AssertionError: True is not false : Received Valid Temp when should have been rejected for invalid auth.

FAIL: test_unit_change (__main__.TestExploits)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/tc/3ljmtg750xz6n4_dy5k9l0mm0000gn/T/ipykernel_63586/781501239.py", line 46, in test_unit_change
    self.assertNotAlmostEqual(deg_k,deg_c,places=0,msg="Reported Temp in K and C should not be the same.")
AssertionError: 74.267842 == 74.268808 within 0 places : Reported Temp in K and C should not be the same.

--------

deg celcius=74.268808
